In [7]:
# Imports de base
import os
import sqlite3
from sqlalchemy import create_engine
from datetime import datetime
import sklearn
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re
import nltk
import json
from keybert import KeyBERT

# Chargement de la configuration
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
with open(os.path.join(PROJECT_ROOT, "config.json")) as f:
    raw_config = json.load(f)

config = {
    key: os.path.normpath(os.path.join(PROJECT_ROOT, value)) if isinstance(value, str) else value
    for key, value in raw_config.items()
}

nltk.download('punkt')

# Initialisation
db_path = config["db_path"]
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# Pour usage SQLAlchemy plus tard
engine = create_engine(f'sqlite:///{db_path}')

# Initialisation KeyBERT
kw_model = KeyBERT()


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/victorcarre/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
# Création des tables
cur.execute('''
    CREATE TABLE IF NOT EXISTS conversations (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_input TEXT,
        llm_output TEXT,
        timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
    )
''')

cur.execute('''
    CREATE TABLE IF NOT EXISTS keywords (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        conversation_id INTEGER,
        keyword TEXT,
        FOREIGN KEY(conversation_id) REFERENCES conversations(id)
    )
''')

conn.commit()


In [9]:
def insert_conversation(user_input, llm_output):
    """
    Insère une nouvelle conversation + mots-clés associés dans la base de données.
    """
    combined_text = user_input + " " + llm_output
    keywords = extract_keywords(combined_text)

    cur.execute("INSERT INTO conversations (user_input, llm_output) VALUES (?, ?)", (user_input, llm_output))
    conversation_id = cur.lastrowid

    for kw in keywords:
        cur.execute("INSERT INTO keywords (conversation_id, keyword) VALUES (?, ?)", (conversation_id, kw))
    
    conn.commit()


In [11]:
def extract_keywords(text, top_n=10):
    """
    Utilise KeyBERT pour extraire des mots-clés pertinents.
    """
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2), stop_words='english', top_n=top_n)
    return [kw for kw, _ in keywords]

    # Nettoyage : suppression des doublons et des stop words restants
    unique_keywords = []
    seen = set()
    for kw, _ in keywords:
        kw_clean = kw.lower().strip()
        # Filtrage des mots très courts, déjà vus, ou présents dans les stopwords
        if (
            kw_clean not in seen and
            kw_clean not in ENGLISH_STOP_WORDS and
            len(kw_clean) > 2 and  # facultatif : éviter les mots très courts
            re.match(r'^[a-zA-Z\-]+$', kw_clean)
        ):
            seen.add(kw_clean)
            unique_keywords.append(kw_clean)
        if len(unique_keywords) >= top_n:
            break

    return unique_keywords

def insert_conversation(user_input, llm_output):
    """
    Insère une conversation dans la base et ses mots-clés associés.
    """
    combined_text = f"{user_input} {llm_output}"
    keywords = extract_keywords(combined_text)

    cur.execute(
        "INSERT INTO conversations (user_input, llm_output) VALUES (?, ?)",
        (user_input, llm_output)
    )
    conversation_id = cur.lastrowid

    for kw in keywords:
        cur.execute(
            "INSERT INTO keywords (conversation_id, keyword) VALUES (?, ?)",
            (conversation_id, kw)
        )
    conn.commit()


In [5]:
def search_conversations_by_keyword(search_term):
    """
    Recherche toutes les conversations associées à un mot-clé donné.
    """
    cur.execute('''
        SELECT c.user_input, c.llm_output, c.timestamp
        FROM conversations c
        JOIN keywords k ON c.id = k.conversation_id
        WHERE k.keyword LIKE ?
        ORDER BY c.timestamp DESC
    ''', (f'%{search_term}%',))

    return cur.fetchall()


In [12]:
# Exemple : insertion
user_input = "Comment fonctionnent les panneaux solaires ?"
llm_output = "Les panneaux solaires transforment la lumière en électricité via l'effet photovoltaïque."

insert_conversation(user_input, llm_output)

# Recherche
results = search_conversations_by_keyword("solaire")
for r in results:
    print("Q:", r[0])
    print("A:", r[1])
    print("⏱️", r[2])
    print("-" * 50)


NameError: name 'search_conversations_by_keyword' is not defined

In [14]:
import pandas as pd

query = '''
SELECT c.id, c.timestamp, c.user_input, c.llm_output, GROUP_CONCAT(k.keyword, ', ') AS keywords
FROM conversations c
LEFT JOIN keywords k ON c.id = k.conversation_id
GROUP BY c.id
ORDER BY c.timestamp DESC
LIMIT 50
'''

df = pd.read_sql_query(query, conn)
df.head(60)


DatabaseError: Execution failed on sql '
SELECT c.id, c.timestamp, c.user_input, c.llm_output, GROUP_CONCAT(k.keyword, ', ') AS keywords
FROM conversations c
LEFT JOIN keywords k ON c.id = k.conversation_id
GROUP BY c.id
ORDER BY c.timestamp DESC
LIMIT 50
': no such column: c.timestamp

In [ ]:
# Exemple de requête SQL

```sql
SELECT c.user_input, c.llm_output, GROUP_CONCAT(k.keyword, ', ') as keywords
FROM conversations c
LEFT JOIN keywords k ON c.id = k.conversation_id
GROUP BY c.id
ORDER BY c.timestamp DESC
LIMIT 10;